In [1]:
from Mediapipe import MediapipeHands
import pandas as pd
import os

initial_dir = os.getcwd()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
mediapipeHands = MediapipeHands(past_data_path="/Users/andreaamaya/Trabajo/megaproyecto_model/data/data.csv")

In [3]:
initial_sequence_id = mediapipeHands.sequence_id
initial_sequence_id

1117

In [4]:
training_dir = f'{initial_dir}/data/videos_lensegua_5'
mediapipeHands.extract_coordinates_from_dir(training_dir)

Hay 633 videos a extraer coordenadas
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/videos_lensegua_5/vendaje (1) copy.mp4  con id  1207


In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua_2'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_2'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_3'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [5]:
validation_dir = f'{initial_dir}/data/videos_val_4'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

Hay 19 videos a extraer coordenadas


In [6]:
df = mediapipeHands.get_padded_data()
validation_sequence_ids = mediapipeHands.validation_sequence_ids

In [7]:
validation_sequence_ids

[1637,
 1638,
 1639,
 1640,
 1641,
 1642,
 1643,
 1644,
 1645,
 1646,
 1647,
 1648,
 1649,
 1650,
 1651,
 1652,
 1653,
 1654,
 1655]

In [8]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/validation.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    
    train_path = f'{initial_dir}/data/train.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    

    if (initial_sequence_id != 0):
        past_data = pd.read_csv(validation_path)
        final_df = pd.concat([past_data, validation_df])
        final_df.to_csv(validation_path, index=False)

        past_data = pd.read_csv(train_path)
        final_df = pd.concat([past_data, train_df])
        final_df.to_csv(train_path, index=False)
    else:
        validation_df.to_csv(validation_path, index=False)
        train_df.to_csv(train_path, index=False)

In [9]:
data_path = f'{initial_dir}/data/data.csv'
df_final = df.drop("file", axis=1)

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df_final])
    final_df.to_csv(data_path, index=False)
else:
    df_final.to_csv(data_path, index=False)

In [ ]:
mediapipeHands = MediapipeHands(past_data_path="/Users/andreaamaya/Trabajo/megaproyecto_model/data/data_tests.csv")

In [ ]:
initial_sequence_id = mediapipeHands.sequence_id
initial_sequence_id

In [ ]:
extra_val_dir = f'{initial_dir}/data/test videos 2'
mediapipeHands.extract_coordinates_from_dir(extra_val_dir)

df = mediapipeHands.get_padded_data()

In [ ]:
data_path = f'{initial_dir}/data/data_tests.csv'

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df])
    final_df.to_csv(data_path, index=False)
else:
    df.to_csv(data_path, index=False)